<div class="alert alert-block alert-danger">
TODO change to the  code from lectures and change  progression:
    <ol> 
    <li>experiment with widget- MCQWs to guide them and test what we want them to get out of it</li>
    <li>  Iris data - making the point that we could see one class was separable so setting up a chain of two-way classifiers </li>
    <li> MNIST - introduction to dataset and then make a classsifier for one digit.<br> adapt code from could show code from https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html#sphx-glr-auto-examples-linear-model-plot-sparse-logistic-regression-mnist-py to visualise the weights in the model- e.g. for 8s</li>
    <li> link to blog https://www.zswarth.com/blog/creating-a-perceptron-classifier-for-the-mnist-dataset/ to read and answer one mcq </li>
    <li> discuss chaining vs parallel and when you might use either: <b>mentimeter to mediate?</b> </li>
    </ol></div>

## Activity1:
Work through the first few cells ‘Logical Operators’, ‘Implementation’ and ‘Linear Decision Boundary’ sections and try to familiarise yourself with the algorithm and how the process builds a model by learning values for the weights.
How well does it perform on different logical functions?
- You do not need to remember the equation for the calculating the line.
- You should try and understand how this decision boundary relates to the Perceptrons output (and why it can only be straight).



# Perceptrons - The basis of Artificial Neural Networks

Perceptrons, invented by Frank Rosenblatt in the late 1950's,
are a form of supervised machine learning algorithm inspired by neuron cells.
In neurons, signals come in along the dendrites and out along the axon. 
A synapse is the connection between the axon of one cell and the dendrites of another.
Crudely, input signals are 'summed' and if they reach a certain threshold the neuron 'fires'
and sends a signal down the synapse to the connected cells.

![Perceptron](resources/Perceptron.png "Perceptron Image")

Perceptrons are an algorithmic approximation of this process and can learn to solve simple classification problems.
Input values are multiplied by a learnable parameter called a *weight*.
If the sum of the inputs $\times$ weights is over a certain threshold the Perceptron 'fires' and generates an output.
We use the *error* in the output to change the value of the *weights* by a small amount - the *learning rate*.
The process is repeated until the error is 0, or as small as we can get it.

**Note:** The threshold which determines if the Perceptron produces an output is determined by its *activation function*.
For Perceptrons this is often a step function which outputs a 1 or 0 i.e. 'fires' or not. However, it can also be a
non-linear function such as sigmoid, which will always produce a real numbered output in the range 0 to 1.

### Perceptron - Algorithm

1. Set weights to random values in range [-0.5, 0.5]

2. Set learning rate to a small value, usually less than 0.5

3. For each training example in the dataset i.e one 'epoch'

    A. Calculate output (activation)
    
    $sum = \sum\limits_{i=0}^{n} w_i \times x_i$
      
    $if\ sum >\ 0 \\ \;\;\;activation = 1 \\ \\else \\ \;\;\;activation = 0$
       
    B. Calculate error
    
    $error = target \, output - activation$

    C. Update each of the weights values
    
    $change \, in \, weight = error \times input \times learning \, rate$


4. Repeat from step 3 until error is 0 (or as close as possible), or for the number of training epochs.

### Perceptrons - Logical Operators

We are going to use binary data to show that Perceptrons can learn to represent logical functions,
though you could also think about it as a prediction/classification problem
i.e. for a given set of inputs what is the correct output.
A truth table can be used as the Perceptrons *training* data, with each row representing an input example.
Each training example has two inputs (*features*) and one output (*label*).

| Input 1| Input 2| AND | OR  | XOR |
|:------:|:------:|:---:|:---:|:---:|
| 0      | 0      | 0   | 0   | 0   | 
| 0      | 1      | 0   | 1   | 1   |
| 1      | 0      | 0   | 1   | 1   |
| 1      | 1      | 1   | 1   | 0   |

First we will import some python modules and then create the training data.

**Note:** Input data is often denoted as X and labels/target outputs with Y.
Here we are going to use **inputs**, but the target outputs have been labeled **AND**, **OR** and **XOR**.
This is so we can be clear about what the outputs should be.

In [ ]:
# Import some needed modules
import plotly
import numpy as np
from IPython.display import display
from perceptron_utils import BinaryPerceptronGraph
plotly.offline.init_notebook_mode(connected=True)

# Create input and target output data
inputs = [[0, 0],
          [1, 0],
          [0, 1],
          [1, 1]] 
print("Input data: " + str(inputs))

target_outputs_AND = [0, 0, 0, 1]
print("AND: " + str(target_outputs_AND))

target_outputs_OR = [0, 1, 1, 1]
print("OR: " + str(target_outputs_OR))

target_outputs_XOR = [0, 1, 1, 0]
print("XOR: " + str(target_outputs_XOR))

### Perceptron - Implementation

Now lets write a function to build and train a Perceptron.
This is just an implementation of the algorithm above, except we are going to train one **step** or one **epoch** at a time.
This allows us to see what the algorithm is doing more clearly.

- A training **step** applies the algorithm to just one input example (A, B and C above).
- An **epoch** repeats the training step for all input examples in the data (so in this case 4).

First we define the learning rate and model.

- The **learning_rate** variable determines how large a change we will make to the weights each time they are updated.

- The **model** is the collection of weight variables that are adjusted ('learned') as the model is trained to produce the 
target outputs. The model will be stored as a python dictionary to keep everything in one data structure.

As it trains you should see output for the current inputs and target outputs,
training step, epoch and total error for that epoch.

In [ ]:
# Set the learning rate
learning_rate = 0.1

# Initialise the model weights to small random values in the range -0.5 to 0.5
weight_1 = np.random.uniform(-0.5, 0.5)
weight_2 = np.random.uniform(-0.5, 0.5)
bias_w = np.random.uniform(-0.5, 0.5)
bias = 1
# Store the model in a dictionary
model = {'weight_1':weight_1, 'weight_2':weight_2, 'bias_weight': bias_w}
print("Initialised model weights:")
print(model)

# Trains the perceptron for one step or epoch
def train(inputs, target_outputs, step, num_steps):
    # Unpack the model values into local variables
    w1, w2, bw = model['weight_1'],  model['weight_2'], model['bias_weight']

    # Define some other variables we need
    weight_sum, activation, error = 0, 0, 0

    # Loop for the desired number of steps (1 step or 1 epoch)
    for i in range(step, num_steps):
        
        # Calculate sum of inputs * weights
        weight_sum = (inputs[i][0] * w1) + (inputs[i][1] * w2) + (bias * bw)

        # Activation (step) function
        if weight_sum > 0:
            activation = 1
        else:
            activation = 0

        # Calculate error (target output - actual output)
        error = target_outputs[i] - activation

        # Update weights (error * input * learning rate)
        w1 += error * inputs[i][0] * learning_rate
        w2 += error * inputs[i][1] * learning_rate
        bw += error * bias * learning_rate
        
        # Update the graph
        perceptron_graph.update_step(model, w1, w2, bw, error)
        # Repack the model values from local variables
        model['weight_1'], model['weight_2'], model['bias_weight'] = w1, w2, bw

# Create the perceptron graph widget
perceptron_graph = BinaryPerceptronGraph()
train_widget = perceptron_graph.create_train_graph(train, learning_rate)
display(train_widget)

### Perceptrons - Linear Decision Boundary

To give you an intuition for what the Perceptron is doing, consider the equation for a straight line:

$y = ax + c$

a and c are coefficients just like the learned weights and bias in the Perceptron.
So with a bit of rearranging:
   
$y = (input1 \times weight1) + (input2 \times weight2) + bias \, weight$

Becomes:

$input2 = (-weight1 \div weight2) \times input1) + (-bias \, weight\div weight2)$

- weights 1 and 2 = slope

- bias = intercept

- step function = which side of the line!

So, the Perceptron is essentially learning a function for a straight line which is called the decision boundary.
In this case, which 'class' the set of inputs belongs to i.e. True or False.

You can run the below code to create an interactive widget that allows you to manually adjust the weights.

In [ ]:
# Create the perceptron decision boundary widget
perceptron_graph = BinaryPerceptronGraph()
decision_boundary_widget = perceptron_graph.create_decision_boundary_graph()
display(decision_boundary_widget)

In [ ]:
TODO add mcq “if a I have a perceptron that does or, what do I change to make to do amd? Increase decrease w1,2,3

# Activity2: 
The aim of this task is to illustrate how Perceptrons can be applied to a larger, more realistic, dataset with real numbered features. We will apply the algorithm in using the training step code from before, but this time we will train the perceptron for a fixed number of epochs and then test the model on a held back test set.
In the ‘Training and Testing on Real-Fake Data’ section the first cell generates some random training and test data.
In the next cell you need to complete the train_perceptron function by copying the relevant code (only the training step!) from previous cells. Once it is complete you should see output of error and accuracy from training.
Next you can complete the test_perceptron function in a similar manner and you should see output of how well the trained model performed on the test data.
What effect does the learning rate and number of examples per epoch have?
Try changing some of the parameters of the randomly created data and see what effect it has.



### Perceptrons - Training and Testing on Real-Fake Data

Truth table data and logical functions are a good way to learn the Perceptron algorithm but the data isn't very realistic.

Most problems are much more complex and cannot be represented with binary data or solved with only 4 training examples.
We were also only training for one **step** (one input example) or one **epoch** (all input examples) at a time, so that we
could see what the algorithm was doing.

In supervised learning, generally we want to train for a fixed number of epochs, or until there is no improvement in
the error on the training data. Once training is finished we apply the model (trained weights) to some test data and
measure its performance. This gives us an indication of how well it would perform on new data it has not 'seen' before.

Next we will train and then test a Perceptron on a larger, real numbered dataset so that we can see the process of 
applying machine learning in practice.

As before, we will first import some python modules and then randomly generate some training and test data.
This time the features of the data will be real numbers but there are still only 2 classes/labels, 0 and 1.

It's also helpful to plot the data so that we can see how it is distributed.

**Note:** The make_blobs function generates a random dataset, the centers variable determines how many classes/labels
are in the data, n_features is the number of features each example has and cluster_std is the standard deviation of
each class i.e. how randomly scattered they are from eachother.

In [ ]:
# Import some needed modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
%matplotlib inline

# Generate random dataset
num_samples = 150
features, labels = make_blobs(n_samples=num_samples, centers=2, n_features=2, cluster_std=1.0, random_state=0)

# Split data to training and test data, 2/3 for training and 1/3 for testing
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.33)

# Print some information about the data
print("Shape of training data: " + str(train_x.shape))
print("Shape of test data: " + str(test_x.shape))
print("First 5 features of training data:")
print(train_x[: 5, :])
print("First 5 labels of training data:")
print(train_y[:5])

# Plot the training data
figure, ax = plt.subplots(1, 2, figsize=(16, 6))
train_plot = sns.scatterplot(x=train_x[:,0], y=train_x[:,1], hue=train_y, ax=ax[0])
ax[0].title.set_text("Train")
test_plot = sns.scatterplot(x=test_x[:,0], y=test_x[:,1], hue=test_y, ax=ax[1])
ax[1].title.set_text("Test")
plt.show()

### Perceptron - Train Function

Now we can define the training function for the Perceptron. It is very similar to the previous, except it will train
for the num_epochs that we define.

The function **inputs** are the training examples, training labels, number of epochs and learning rate.

The function **returns** the trained model i.e. the 3 trained weights.

Every epoch will calculate and print the error and accuracy for that epoch.

Once training is complete we can use the equation from the *Linear Decision Boundary* section above to draw the
decision boundary for the trained model.

In [ ]:
# Define the number of training epochs and learning rate
num_epochs = 10
learning_rate = 0.1

def train_perceptron(inputs, target_outputs, num_epochs, learning_rate):

    # Set the weights to small random values in the range -0.5 to 0.5  
    w1 = np.random.uniform(-0.5, 0.5)
    w2 = np.random.uniform(-0.5, 0.5)
    bw = np.random.uniform(-0.5, 0.5)
    bias = 1
    print("Initialised model weights:")
    print("weight_1: " + str(w1) + " weight_2: " + str(w2) + " bias_weight: " + str(bw))
    
    # Each epoch will loop over the training data once
    for epoch in range(num_epochs):
        epoch_error = 0

        # Loop over all of the input examples
        for i in range(len(inputs)):
            error = -1
            # Calculate sum of inputs * weights

            
            # Activation (step) function


            # Calculate error (desired output - actual output)

            epoch_error += np.absolute(error) # Also keep track of total error for this epoch

            # Update weights (error * input * learning rate)

        
        # Calculate epoch accuracy and print metrics    
        epoch_accuracy = (100/len(inputs)) * (len(inputs) - epoch_error)
        print("Epoch: " + str(epoch + 1) + " Error: " + str(epoch_error) + " Accuracy: " + str(round(epoch_accuracy, 2)))
    return w1, w2, bw

# Call the function to train the Perceptron and return the trained weights
input_w1, input_w2, bias_w = train_perceptron(train_x, train_y, num_epochs, learning_rate)

print("Trained model weights:")
print("w1: " + str(input_w1) + " w2: " + str(input_w2) + " bw: " + str(bias_w))

# Create the range of values for decision boundary
x_range = np.linspace(train_x[:,0].min(), train_x[:,0].max(), 10)
y_range = [((-input_w1/input_w2) * x) + (-bias_w/input_w2) for x in x_range]

# Plot the training data and the decision boundary
figure, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(x=train_x[:,0], y=train_x[:,1], hue=train_y, ax=ax)
sns.lineplot(x=x_range, y=y_range, color='r', ax=ax)
plt.show()

### Perceptron - Test Function

Now we can test the trained model by making predictions on the test data.

The main difference is that now we **don't** update the weights. So, we just pass in the trained weights and iterate
for one epoch over the training data, recording the error and accuracy.

Finally we can plot the test data and decision boundary as before.

In [ ]:
def test_perceptron(inputs, target_outputs, w1, w2, bw):
    
    # Keep track of the error on the test data
    test_error = 0
    
    # Loop over all of the test examples
    for i in range(len(inputs)):
        error = -1



        test_error += np.absolute(error) # Also keep track of total error testing
      
    # Calculate epoch accuracy and print metrics    
    test_accuracy = (100/len(inputs)) * (len(inputs) - test_error)
    print("Test Error: " + str(test_error) + " Accuracy: " + str(round(test_accuracy, 2)))

# Call the test perceptron function with the test data and trained model    
test_perceptron(test_x, test_y, input_w1, input_w2, bias_w)

# Create the range of values for decision boundary
x_range = np.linspace(test_x[:,0].min(), test_x[:,0].max(), 10)
y_range = [((-input_w1/input_w2) * x) + (-bias_w/input_w2) for x in x_range]

# Plot the training data and the decision boundary
figure, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(x=test_x[:,0], y=test_x[:,1], hue=test_y, ax=ax)
sns.lineplot(x=x_range, y=y_range, color='r', ax=ax)
plt.show()